In [11]:
import pandas as pd
import numpy as np
import csv
import pyodbc
import os

In [19]:
# Filter 'mode' to only include 'StateAssignedSchoolID' that exist in 'main'
mode = mode[mode['StateAssignedSchoolID'].isin(main['schoolcode'])]

# Subset 'main' for year 2021 and merge with 'mode'
main_2021 = main[main['year'] == 2021]
merged_2021 = pd.merge(main_2021, mode, left_on='schoolcode', right_on='StateAssignedSchoolID', how='left')

# Subset 'main' for other years and add columns from 'mode' filled with zeros
main_other_years = main[main['year'] != 2021]
for col in mode.columns:
    if col not in main_other_years.columns:
        main_other_years[col] = 0

# Concatenate both parts
final_data = pd.concat([merged_2021, main_other_years], ignore_index=True)

C:\Users\natan\AppData\Local\Temp\ipykernel_20668\2531846427.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_other_years[col] = 0
C:\Users\natan\AppData\Local\Temp\ipykernel_20668\2531846427.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_other_years[col] = 0
C:\Users\natan\AppData\Local\Temp\ipykernel_20668\2531846427.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [20]:
final_data

,Unnamed: 0_x,schoolcode,totalenroll,whiteenroll,blackenroll,hispanicenroll,lowincomeenroll,attendance,mobility,dropout,...,retention,year,schoolmode,Unnamed: 0_y,StateAssignedSchoolID,Closed,Hybrid,In-person,Virtual,Unnamed: 0
0,0.0,11012002C260001,360.0,94.7,0.0,0.0,27.2,93.0,8.6,3.1,...,92.3,2021,0.000000,70.0,11012002C260001,0.090909,0.000000,0.909091,0.000000,NaN
1,3.0,110150010260001,756.0,85.1,5.0,4.0,43.7,90.0,10.3,5.5,...,87.4,2021,0.227273,78.0,110150010260001,0.000000,0.454545,0.545455,0.000000,NaN
2,6.0,110150020260001,921.0,86.4,3.0,5.1,54.6,90.5,11.2,3.1,...,90.5,2021,0.363636,84.0,110150020260001,0.090909,0.363636,0.363636,0.181818,NaN
3,9.0,110150050260002,72.0,90.3,0.0,0.0,41.7,93.7,10.2,6.9,...,84.8,2021,0.000000,88.0,110150050260002,0.090909,0.000000,0.909091,0.000000,NaN
4,12.0,110180770260001,292.0,97.3,0.0,0.0,36.6,94.3,8.6,1.0,...,96.8,2021,0.363636,92.0,110180770260001,0.090909,0.727273,0.181818,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,NaN,56099365U260007,3550.0,19.5,26.6,41.8,59.4,89.0,8.5,2.7,...,90.0,2018,0.000000,NaN,0,0.000000,0.000000,0.000000,0.000000,1454.0
1457,NaN,56099365U260008,1869.0,33.9,12.5,45.9,57.7,92.9,8.0,3.7,...,92.9,2019,0.000000,NaN,0,0.000000,0.000000,0.000000,0.000000,1456.0
1458,NaN,56099365U260008,1920.0,34.7,12.7,44.8,59.1,90.6,5.0,1.6,...,91.3,2018,0.000000,NaN,0,0.000000,0.000000,0.000000,0.000000,1457.0
1459,NaN,651089010800001,615.0,74.8,5.4,6.3,1.8,0.0,1.9,1.0,...,88.1,2019,0.000000,NaN,0,0.000000,0.000000,0.000000,0.000000,1459.0


In [22]:
final_data = final_data.drop(columns=['StateAssignedSchoolID', 'Unnamed: 0_y', 'Unnamed: 0_x'])

In [23]:
final_data = final_data.rename(columns={                   
                        'Virtual': 'virtual_per',    
                        'Hybrid': 'hybrid_per',     
                                       })
final_data

,schoolcode,totalenroll,whiteenroll,blackenroll,hispanicenroll,lowincomeenroll,attendance,mobility,dropout,graduation,classsize,retention,year,schoolmode,Closed,hybrid_per,In-person,virtual_per,Unnamed: 0
0,11012002C260001,360.0,94.7,0.0,0.0,27.2,93.0,8.6,3.1,89.7,16.4,92.3,2021,0.000000,0.090909,0.000000,0.909091,0.000000,NaN
1,110150010260001,756.0,85.1,5.0,4.0,43.7,90.0,10.3,5.5,72.1,22.2,87.4,2021,0.227273,0.000000,0.454545,0.545455,0.000000,NaN
2,110150020260001,921.0,86.4,3.0,5.1,54.6,90.5,11.2,3.1,88.2,20.5,90.5,2021,0.363636,0.090909,0.363636,0.363636,0.181818,NaN
3,110150050260002,72.0,90.3,0.0,0.0,41.7,93.7,10.2,6.9,94.1,11.0,84.8,2021,0.000000,0.090909,0.000000,0.909091,0.000000,NaN
4,110180770260001,292.0,97.3,0.0,0.0,36.6,94.3,8.6,1.0,95.9,15.4,96.8,2021,0.363636,0.090909,0.727273,0.181818,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,56099365U260007,3550.0,19.5,26.6,41.8,59.4,89.0,8.5,2.7,86.0,19.0,90.0,2018,0.000000,0.000000,0.000000,0.000000,0.000000,1454.0
1457,56099365U260008,1869.0,33.9,12.5,45.9,57.7,92.9,8.0,3.7,92.8,22.0,92.9,2019,0.000000,0.000000,0.000000,0.000000,0.000000,1456.0
1458,56099365U260008,1920.0,34.7,12.7,44.8,59.1,90.6,5.0,1.6,92.6,20.0,91.3,2018,0.000000,0.000000,0.000000,0.000000,0.000000,1457.0
1459,651089010800001,615.0,74.8,5.4,6.3,1.8,0.0,1.9,1.0,97.2,22.1,88.1,2019,0.000000,0.000000,0.000000,0.000000,0.000000,1459.0


In [24]:
final_data.to_csv("final_drop.csv")